# Sentiment analysis Twitter Airline dataset with optimized NRC algorithm

In [ ]:
import pandas as pd
import numpy as np
import dill
import nltk

In [ ]:
nltk.download()

In [ ]:
dill.dump_session('notebook_env.db')

In [ ]:
dill.load_session('notebook_env.db')

In [2]:
dataset = pd.read_csv('twitter-airline-sentiment.csv')
dataset

tweet_id airline_sentiment  airline_sentiment_confidence  \
0      570306133677760513           neutral                        1.0000   
1      570301130888122368          positive                        0.3486   
2      570301083672813571           neutral                        0.6837   
3      570301031407624196          negative                        1.0000   
4      570300817074462722          negative                        1.0000   
5      570300767074181121          negative                        1.0000   
6      570300616901320704          positive                        0.6745   
7      570300248553349120           neutral                        0.6340   
8      570299953286942721          positive                        0.6559   
9      570295459631263746          positive                        1.0000   
10     570294189143031808           neutral                        0.6769   
11     570289724453216256          positive                        1.0000   
12     570289584061480960          positive                        1.0000   
13     570287408438120448          positive                        0.6451   
14     570285904809598977          positive                        1.0000   
15     570282469121007616          negative                        0.6842   
16     570277724385734656          positive                        1.0000   
17     570276917301137409          negative                        1.0000   
18     570270684619923457          positive                        1.0000   
19     570267956648792064          positive                        1.0000   
20     570265883513384960          negative                        0.6705   
21     570264145116819457          positive                        1.0000   
22     570259420287868928          positive                        1.0000   
23     570258822297579520           neutral                        1.0000   
24     570256553502068736          negative                        1.0000   
25     570249102404923392          negative                        1.0000   
26     570239632807370753          negative                        1.0000   
27     570217831557677057           neutral                        0.6854   
28     570207886493782019          negative                        1.0000   
29     570124596180955136           neutral                        0.6150   
...                   ...               ...                           ...   
14610  569591765793165312          negative                        1.0000   
14611  569591730506371072           neutral                        1.0000   
14612  569591700416393216          negative                        1.0000   
14613  569591653121597440          negative                        1.0000   
14614  569591540944756737          negative                        1.0000   
14615  569591533617307648          negative                        1.0000   
14616  569591393540288512          negative                        1.0000   
14617  569591285150908416          positive                        1.0000   
14618  569591136534319105          negative                        1.0000   
14619  569590988395708416          positive                        1.0000   
14620  569590965880532993          negative                        1.0000   
14621  569590892085915649          negative                        1.0000   
14622  569590191758962688          negative                        1.0000   
14623  569590013278756865          positive                        0.6274   
14624  569589959088173056          negative                        1.0000   
14625  569589643487928321          positive                        1.0000   
14626  569589460226183168          negative                        1.0000   
14627  569588816438169600          negative                        1.0000   
14628  569588651925098496          positive                        1.0000   
14629  569588591602458624          negative                        1.0000   
14630  569588

Split dataset in positive - negative - neutral

In [3]:
positive = dataset.query('airline_sentiment == "positive"')['text']
negative = dataset.query('airline_sentiment == "negative"')['text']
neutral = dataset.query('airline_sentiment == "neutral"')['text']

Statistics

In [4]:
print('# Positives: ' + str((len(positive))))
print('# Negatives: ' + str((len(negative))))
print('# Neutral: ' + str((len(neutral))))

# Positives: 2363
# Negatives: 9178
# Neutral: 3099


Very unbalanced classes. take the class with less samples (positives) and makes it the "benchmark", then split into train and test data

In [5]:
np_positive = positive.values
np_negative = negative.values
np_neutral = neutral.values

# shuffle
np.random.seed(10)
np.random.shuffle(np_positive)
np.random.shuffle(np_negative)
np.random.shuffle(np_neutral)

# take the same number of sentences
np_positive = np_positive[:min(len(np_positive), len(np_negative), len(np_neutral))]
np_negative = np_negative[:min(len(np_positive), len(np_negative), len(np_neutral))]
np_neutral = np_neutral[:min(len(np_positive), len(np_negative), len(np_neutral))]

Preprocessing. Just TF-IDF with word representation

In [6]:
import re
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn

def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)
    return tweet

def preprocess_word(word):
    # remove punctation
    word = re.sub('[\'"?!,.():;*]+', '', word)#word.strip('\'"?!,.():;*')
    # more than 3 letter repetition removed
    word = re.sub(r'(.)\1\1+', r'\1\1\1', word)
    # remove - & '
    word = word.strip('-&\'')
    return word

def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)

def n_upper_words(tweet):
    n = 0
    for word in tweet.split():
        if word.isupper():
            n+=1
    return n

def n_hashtags(tweet):
    n = 0
    for word in tweet.split():
        if re.search(r'^#[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None:
            n+=1
    return n

def punctation_sequences(tweet):
    qm_seq = 0
    em_seq = 0
    qm_em_seq = 0
    qm = re.findall(r'[^\!][\?][\?]+($|\s|\n)|^[\?][\?]+($|\s|\n)', tweet)
    if qm is not None:
        qm_seq = len(qm)
    em = re.findall(r'[^\?][\!][\!]+($|\s|\n)|^[\!][\!]+($|\s|\n)', tweet)
    if qm is not None:
        em_seq = len(em)
    qm_em = re.findall(r'(\?)+[\!]+[\!\?]*|(\!)+[\?]+[\!\?]*', tweet)
    if qm_em is not None:
        qm_em_seq = len(qm_em)
    
    return qm_seq, em_seq, qm_em_seq

# 1 if last token contains question mark or exclamation mark. 0 otherwise
def last_token_punctation(tweet):
    last_qm = 0
    last_em = 0
    tokens = tweet.split()
    last_token = tokens[len(tokens)-1]
    if re.findall(r'\?', last_token) is not None:
        last_qm = 1
    if re.findall(r'\!', last_token) is not None:
        last_qm = 1
    return last_qm, last_em

# 1 if contains positive emoticon, 0 otherwise (same for negative emoticons)
def presence_emoticon(tweet):
    pres_pos = 0
    pres_neg = 0
    if re.findall(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))|(:\s?D|:-D|x-?D|X-?D)|(<3|:\*)|(;-?\)|;-?D|\(-?;)', tweet) is not None:
        pres_pos = 1
    if re.findall(r'(:\s?\(|:-\(|\)\s?:|\)-:)|(:,\(|:\'\(|:"\()', tweet) is not None:
        pres_neg = 1
        
    return pres_neg, pres_pos

# 1 if last token is a positive emoticon, 0 otherwise (same for negative emoticon)
def last_token_emoticon(tweet):
    last_pos = 0
    last_neg = 0
    tokens = tweet.split()
    last_token = tokens[len(tokens)-1]
    if re.findall(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))|(:\s?D|:-D|x-?D|X-?D)|(<3|:\*)|(;-?\)|;-?D|\(-?;)', last_token) is not None:
        last_pos = 1
    if re.findall(r'(:\s?\(|:-\(|\)\s?:|\)-:)|(:,\(|:\'\(|:"\()', last_token) is not None:
        last_neg = 1
    return last_neg, last_pos

# get the number of elongated words (one character repeated > 2 times)
def n_elongated(tweet):
    n_elong = 0
    tokens = tweet.split()
    for t in tokens:
        n_elong += len(re.findall(r'([a-zA-Z])\1{2,}', t))
    return n_elong

def word_scores(word):
    # take the mean of the scores for each class
    neg_score = 0
    obj_score = 0
    pos_score = 0
    list_swnt = list(swn.senti_synsets(word))
    if len(list_swnt) == 0:
        return None
    for word in list_swnt:
        neg_score += word.neg_score()
        obj_score += word.obj_score()
        pos_score += word.pos_score()
    return neg_score/len(list_swnt), obj_score/len(list_swnt), pos_score/len(list_swnt)

def tot_scores_word(tweet):
    n_neg = 0
    n_obj = 0
    n_pos = 0
    for word in tweet.split():
        if word_scores(word) is not None:
            if word_scores(word)[2] > 0:
                n_pos += 1
            if word_scores(word)[1] > 0:
                n_obj += 1
            if word_scores(word)[0] > 0:
                n_neg += 1
    return n_neg, n_obj, n_pos

def tot_score(tweet):
    score_neg = 0
    score_obj = 0
    score_pos = 0
    for word in tweet.split():
        if word_scores(word) is not None:
            score_pos += word_scores(word)[2]
            score_obj += word_scores(word)[1] 
            score_neg += word_scores(word)[0]
    return score_neg, score_obj, score_pos

def max_scores(tweet):
    max_neg = 0
    max_obj = 0
    max_pos = 0
    for word in tweet.split():
        if word_scores(word) is not None:
            curr_pos = word_scores(word)[2]
            curr_obj = word_scores(word)[1] 
            curr_neg = word_scores(word)[0]
            if curr_pos > max_pos:
                max_pos = curr_pos
            if curr_obj > max_obj:
                max_obj = curr_obj
            if curr_neg > max_neg:
                max_neg = curr_neg
    return max_neg, max_obj, max_pos

def last_score_word(tweet):
    last_pos = 0
    last_obj = 0
    last_neg = 0
    split_tweet = tweet.split()
    for i in range(len(split_tweet)-1, -1, -1):
        word = split_tweet[i]
        if word_scores(word) is not None and word_scores(word)[2] > 0:
            last_pos = word_scores(word)[2]
            break
    for i in range(len(split_tweet)-1, -1, -1):
        word = split_tweet[i]
        if word_scores(word) is not None and word_scores(word)[1] > 0:
            last_obj = word_scores(word)[1]
            break
    for i in range(len(split_tweet)-1, -1, -1):
        word = split_tweet[i]
        if word_scores(word) is not None and word_scores(word)[0] > 0:
            last_neg = word_scores(word)[0]
            break
    return last_neg, last_obj, last_pos
            
def handle_negations(tweet):
    tokens = tweet.split()
    i = 0
    while i < len(tokens):
        curr = tokens[i].lower()
        if curr == 'no' or curr == 'not' or curr == 'none'\
            or curr == 'nobody' or curr == 'nothing'  \
            or curr == 'neither' or curr == 'nowhere' \
            or curr == 'never' or curr == 'nothing' or 'n\'t' in curr:
            while i < len(tokens) and not('.' in tokens[i] or ',' in tokens[i] or ';' in tokens[i] or '!' in tokens[i] or '?' in tokens[i]):
                tokens[i] = tokens[i] + '_NEG'
                i += 1
            # also handle word with punctation
            if i < len(tokens):
                tokens[i] = tokens[i] + '_NEG'
        i += 1
    return " ".join(tokens)

def n_negated_contexts(tweet):
    n_neg = 0
    tokens = tweet.split()
    i = 0
    while i < len(tokens):
        curr = tokens[i].lower()
        if curr == 'no' or curr == 'not' or curr == 'none'\
            or curr == 'nobody' or curr == 'nothing'  \
            or curr == 'neither' or curr == 'nowhere' \
            or curr == 'never' or curr == 'nothing' or 'n\'t' in curr:
            n_neg += 1
            while i < len(tokens) and not('.' in tokens[i] or ',' in tokens[i] \
                    or ';' in tokens[i] or '!' in tokens[i] or '?' in tokens[i] \
                    or tokens[i] == 'no' or tokens[i] == 'not' or tokens[i] == 'none'\
                    or tokens[i] == 'nobody' or tokens[i] == 'nothing'  \
                    or tokens[i] == 'neither' or tokens[i] == 'nowhere' \
                    or tokens[i] == 'never' or tokens[i] == 'nothing' or 'n\'t' in tokens[i]):
                i += 1
        i += 1
    return n_neg

def tweet_features(tweet):
    # number uppercase words
    n_upper = n_upper_words(tweet)
    # number hashtags
    n_hash = n_hashtags(tweet)
    # total count of tokens in the tweet with score(w,p)>0
    n_neg, n_obj, n_pos = tot_scores_word(tweet)
    # total score = sum(score(w,p))
    score_neg, score_obj, score_pos = tot_score(tweet)
    # maximal score = max(score(w,p))
    max_neg, max_obj, max_pos = max_scores(tweet)
    # score of last token with score(w,p)>0
    last_neg, last_obj, last_pos = last_score_word(tweet)
    # number of contiguous sequence of quest mark, esc mark and both
    qm_seq, em_seq, qm_em_seq = punctation_sequences(tweet)
    # wheather last token contains esclamation or question mark
    last_qm, last_em = last_token_punctation(tweet)
    # presence or absence of positive and negative emoticons
    pres_neg, pres_pos = presence_emoticon(tweet)
    # wheather last token is positive or negative emoticon
    last_neg_em, last_pos_em = last_token_emoticon(tweet)
    # number of elongated words
    n_elong = n_elongated(tweet)
    # number of negated contexts
    n_neg = n_negated_contexts(tweet)
    
    return [n_upper, n_hash, n_neg, n_obj, n_pos, score_neg, \
            score_obj, score_pos, max_neg, max_obj, max_pos, \
            last_neg, last_obj, last_pos, qm_seq, em_seq, qm_em_seq, \
            last_qm, last_em, pres_neg, pres_pos, last_neg_em, last_pos_em, \
            n_elong, n_neg]

def preprocess_tweet(tweet, use_stemmer=False, use_lemmatizer=False):
    # convert tweet to lowercase
    tweet = tweet.lower()
    # replace urls with 'URL'
    tweet = re.sub(r'((www.[\S]+)|(https?://.[\S]+))', 'URL', tweet)
    # replace user mentions @user with 'USER_MENTION'
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    # replace #hashtag with hastag
    tweet = re.sub(r'#(\S+)', r' \1', tweet)
    # remove retweet RT
    tweet = re.sub(r'\brt\b', '', tweet)
    # replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # remove space, " and ' 
    tweet.strip('" \'')
    # handle emojis. Use only EMO_POS and EMO_NEG
    tweet = handle_emojis(tweet)
    # replace multiple spaces with only one space
    tweet = re.sub(r'\s+', ' ', tweet)
    
    # handle negations
    tweet = handle_negations(tweet)
    # preprocess words
    words = tweet.split()

    processed_words = []
    porter_stemmer = PorterStemmer()
    wordnet_lemmatizer = WordNetLemmatizer()
    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word):
            if use_stemmer:
                # use stemmer
                word = str(porter_stemmer.stem(word))
            elif use_lemmatizer:
                word = str(wordnet_lemmatizer.lemmatize(word))
            processed_words.append(word)
    return ' '.join(processed_words)


In [ ]:
#punctation_sequences(np_positive[0])
#tweet_features(np_positive[0])

Example of preprocessing

In [ ]:
print('######## POSITIVE ########')
print(np_positive[0])
print(preprocess_tweet(np_positive[0]))
print('######## NEGATIVE ########')
print(np_negative[0])
print(preprocess_tweet(np_negative[0]))
print('######## NEUTRAL ########')
print(np_neutral[0])
print(preprocess_tweet(np_neutral[0]))

Preprocess the whole dataset

In [7]:
# positive
np_positive_preprocess = []
for tweet in np_positive:
    np_positive_preprocess.append(preprocess_tweet(tweet))
# negative
np_negative_preprocess = []
for tweet in np_negative:
    np_negative_preprocess.append(preprocess_tweet(tweet))
# neutral
np_neutral_preprocess = []
for tweet in np_neutral:
    np_neutral_preprocess.append(preprocess_tweet(tweet))

Feature vectors

In [10]:
from nltk import FreqDist
from scipy.sparse import lil_matrix
from sklearn.feature_extraction.text import TfidfTransformer

class FeatureVectorGenerator():
    
    def __init__(self, max_words, max_bigrams):
        # max_words = number of different words considered
        # max_bigramd = num of different bigrams considered
        self.all_words = []
        self.all_bigrams = []
        self.MAX_WORDS = max_words
        self.MAX_BIGRAMS = max_bigrams
        self.VOCAB_SIZE = self.MAX_WORDS + self.MAX_BIGRAMS

    def get_features(self, tweet, words_set, bigrams_set):
        # words_set = set of different words
        # bigram_set = set of different bigrams
        uni_feat = []
        big_feat = []
        words = tweet.split()
        # take just words in words_set
        # and bigrams in bigrams_set
        for i in range(len(words)-1):
            if words[i] in words_set:
                uni_feat.append(words[i])
            if (words[i], words[i+1]) in bigrams_set:
                big_feat.append((words[i], words[i+1]))
        if len(words) >= 1 and words[len(words)-1] in words_set:
            uni_feat.append(words[len(words)-1])
        return uni_feat, big_feat

    def extract_features(self, tweets, test=False):
        # tweets = tweets not yet preprocessed
        INITIAL_FEATURE_SIZE = 25
        # bag of words: 1 if unigram/bigram present, 0 otherwise
        features = lil_matrix((len(tweets), self.VOCAB_SIZE + INITIAL_FEATURE_SIZE))
        for j, tweet in enumerate(tweets):
            # calculate initial features
            init_features = tweet_features(tweet)
            for idx in range(len(init_features)):
                features[j, idx] = init_features[idx]
            # preprocess
            tweet = preprocess_tweet(tweet)
            unigrams, bigrams = self.get_features(tweet, self.all_words_set, self.all_bigrams_set)
            for word in unigrams:
                idx = self.dict_word.get(word)
                if idx:
                    features[j, idx + INITIAL_FEATURE_SIZE] = 1
            for bigram in bigrams:
                idx = self.dict_bigram.get(bigram)
                if idx:
                    features[j, self.MAX_WORDS + idx + INITIAL_FEATURE_SIZE] = 1
        '''
        # apply tf-idf
        if not test:
            transformer = TfidfTransformer(smooth_idf=True, sublinear_tf=True, use_idf=True)
            self.tfidf = transformer.fit(features)
        features = self.tfidf.transform(features)
        '''
        return features
    
    def fit(self, train_preprocessed_tweets):
        # Count the words and calculate the frequency distribution
        for tweet in train_preprocessed_tweets:
            split = tweet.split()
            for i in range(len(split)-1):
                w = split[i]
                b = (split[i], split[i+1])
                self.all_words.append(w)
                self.all_bigrams.append(b)
            if len(split) >= 1:
                self.all_words.append(split[len(split)-1])
        # Different words
        self.all_words_set = set(self.all_words)
        self.all_bigrams_set = set(self.all_bigrams)
        # Some counts
        print('Number total words: ' + str(len(self.all_words)))
        print('Number total bigrams: ' + str(len(self.all_bigrams)))
        print('Number different words: ' + str(len(self.all_words_set)))
        print('Number different bigrams: ' + str(len(self.all_bigrams_set)))
        # Calculate frequencies
        self.word_freq_dist = FreqDist(self.all_words)
        self.bigram_freq_dist = FreqDist(self.all_bigrams)
        # take the most common words and bigrams
        self.dict_word = {x[0] : i for i, x in enumerate(self.word_freq_dist.most_common(self.MAX_WORDS))}
        self.dict_bigram = {x[0] : i for i, x in enumerate(self.bigram_freq_dist.most_common(self.MAX_BIGRAMS))}

Split datset into train and test

In [82]:
# 0 = neutral,  1 = negative,  2 = positive
TRAIN_SIZE = 2000
train_x = []
train_y = []
test_x = []
test_y = []
#train_x.extend(np_positive_preprocess[:TRAIN_SIZE])
train_x.extend(np_positive[:TRAIN_SIZE])
train_x.extend(np_negative[:TRAIN_SIZE])
train_x.extend(np_neutral[:TRAIN_SIZE])
test_x.extend(np_positive[TRAIN_SIZE:])
test_x.extend(np_negative[TRAIN_SIZE:])
test_x.extend(np_neutral[TRAIN_SIZE:])
train_y.extend(np.ones(TRAIN_SIZE) * 2)
train_y.extend(np.ones(TRAIN_SIZE) * 1)
train_y.extend(np.ones(TRAIN_SIZE) * 0)
test_y.extend(np.ones(len(np_positive_preprocess) - TRAIN_SIZE) * 2)
test_y.extend(np.ones(len(np_negative_preprocess) - TRAIN_SIZE) * 1)
test_y.extend(np.ones(len(np_neutral_preprocess) - TRAIN_SIZE) * 0)

# Shuffle train and test datasets
np.random.seed(10)
np.random.shuffle(train_x)
np.random.seed(10)
np.random.shuffle(train_y)
np.random.seed(10)
np.random.shuffle(test_x)
np.random.seed(10)
np.random.shuffle(test_y)
# also shuffle in the same way the initial features
# np.random.seed(10)
# np.random.shuffle(train_feat)
# np.random.seed(10)
# np.random.shuffle(test_feat)

In [83]:
fvg = FeatureVectorGenerator(max_words = 3000, max_bigrams = 4000)
fvg.fit(train_x)
f_train = fvg.extract_features(train_x, test=False)
f_test = fvg.extract_features(test_x, test=True)

Number total words: 96405
Number total bigrams: 90405
Number different words: 16347
Number different bigrams: 56120


In [84]:
# features example
print(f_train[1])

  (0, 0)	2.0
  (0, 3)	10.0
  (0, 4)	5.0
  (0, 5)	0.2464375901875902
  (0, 6)	9.348439754689755
  (0, 7)	0.4051226551226551
  (0, 8)	0.0625
  (0, 9)	1.0
  (0, 10)	0.2840909090909091
  (0, 11)	0.022727272727272728
  (0, 12)	0.6931818181818182
  (0, 13)	0.2840909090909091
  (0, 17)	1.0
  (0, 19)	1.0
  (0, 20)	1.0
  (0, 21)	1.0
  (0, 22)	1.0
  (0, 28)	1.0
  (0, 31)	1.0
  (0, 50)	1.0
  (0, 52)	1.0
  (0, 58)	1.0
  (0, 79)	1.0
  (0, 94)	1.0
  (0, 95)	1.0
  (0, 136)	1.0
  (0, 241)	1.0
  (0, 456)	1.0
  (0, 1293)	1.0
  (0, 3034)	1.0
  (0, 3189)	1.0
  (0, 3213)	1.0
  (0, 3342)	1.0
  (0, 3862)	1.0
  (0, 4842)	1.0


Train SVM model

In [85]:
from sklearn import svm
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV
from time import time
import pprint
from scipy.stats import randint
from scipy.stats import uniform
from skopt.space import Real, Categorical, Integer

In [86]:
def report_perf(optimizer, X, y, title):
    """
    A wrapper for measuring time and performances of different optmizers
    
    optimizer = a sklearn or a skopt optimizer
    X = the training set 
    y = our target
    title = a string label for the experiment
    """
    start = time()
    optimizer.fit(X, y)
    best_score = optimizer.best_score_
    best_score_std = optimizer.cv_results_['std_test_score'][optimizer.best_index_]
    best_params = optimizer.best_params_
    print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
           +u"\u00B1"+" %.3f") % (time() - start, 
                                  len(optimizer.cv_results_['params']),
                                  best_score,
                                  best_score_std))
    print('Best parameters:')
    pprint.pprint(best_params)
    print()
    return best_params

In [87]:
# naive svm classifier
clf_svm = svm.SVC(max_iter=1000, kernel='linear')
clf_svm.fit(f_train, train_y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=1000, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [88]:
# evaluate naive model
svm_prediction = clf_svm.predict(f_test)
accuracy_score(test_y, svm_prediction)

0.5959595959595959

Now try to improve the model estimating hyperparameters

In [91]:
# Converting average precision score into a scorer suitable for model selection
acc_scr = make_scorer(accuracy_score)

In [92]:
# Setting a 5-fold stratified cross-validation (note: shuffle=True)
skf = StratifiedKFold(n_splits=5, shuffle=False, random_state=0)

In [95]:
clf_svm = svm.SVC(max_iter=1000)

Grid Search

In [99]:
# GridSearchCV needs a predefined plan of the experiments
grid_search = GridSearchCV(clf_svm, 
                           param_grid={"C": [0.1, 1,],
                                       "kernel": ["linear", "poly", "rbf", "sigmoid"],
                                       "degree": [2, 3, 5],
                                       "decision_function_shape": ["ovo", "ovr"],
                                       },
                           n_jobs=-1,
                           cv=skf,
                           scoring=acc_scr,
                           iid=False, # just return the average score across folds
                           return_train_score=False,
                           verbose=10)

best_params = report_perf(grid_search, f_train, train_y, 'GridSearchCV')

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   16.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   33.3s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   53.0s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  4

GridSearchCV took 355.68 seconds,  candidates checked: 48, best CV score: 0.739 ± 0.013
Best parameters:
{'C': 0.1, 'decision_function_shape': 'ovo', 'degree': 2, 'kernel': 'linear'}



Random Search

In [100]:
# RandomizedSearchCV needs the distribution of the experiments to be tested
# If you can provide the right distribution, the sampling will lead to faster and better results.
random_search = RandomizedSearchCV(clf_svm, 
                                   param_distributions={"C": uniform(0.1, 10),
                                                        "kernel": ["linear", "poly", "rbf", "sigmoid"],
                                                        "degree": [2, 3, 5],
                                                        "decision_function_shape": ["ovo", "ovr"]
                                                       },
                                   n_iter=100,
                                   n_jobs=-1,
                                   cv=skf,
                                   scoring=acc_scr,
                                   iid=False, # just return the average score across folds
                                   return_train_score=False,
                                   random_state=0,
                                   verbose=10)

best_params = report_perf(random_search, f_train, train_y, 'RandomizedSearchCV')

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   27.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   38.7s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   50.2s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  4

RandomizedSearchCV took 757.75 seconds,  candidates checked: 100, best CV score: 0.734 ± 0.013
Best parameters:
{'C': 0.19356704856532617,
 'decision_function_shape': 'ovo',
 'degree': 2,
 'kernel': 'linear'}



Bayesian Search

In [101]:
search_spaces = {"C": Real(0.01, 1.0),
                 "kernel": ["linear", "poly", "rbf", "sigmoid"],
                 "degree": [2, 3, 5],
                 "decision_function_shape": ["ovo", "ovr"]
                }

for baseEstimator in ['GP', 'RF', 'ET', 'GBRT']:
    opt = BayesSearchCV(clf_svm,
                        search_spaces,
                        scoring=acc_scr,
                        cv=skf,
                        n_iter=40,
                        n_jobs=-1,
                        return_train_score=False,
                        optimizer_kwargs={'base_estimator': baseEstimator},
                        random_state=4,
                        verbose=10)
    
    best_params = report_perf(opt, f_train, train_y,'BayesSearchCV_'+baseEstimator)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   11.4s remaining:   17.1s
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:   11.4s remaining:    7.5s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   11.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   11.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    7.6s remaining:   11.5s
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    7.9s remaining:    5.2s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    8.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    8.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    6.6s remaining:   10.0s
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    6.8s remaining:    4.5s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    6.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    6.9s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    7.6s remaining:   11.5s
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    7.7s remaining:    5.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    8.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    8.0s finished


BayesSearchCV_GBRT took 340.90 seconds,  candidates checked: 40, best CV score: 0.740 ± 0.014
Best parameters:
{'C': 0.11179663308090047,
 'decision_function_shape': 'ovr',
 'degree': 5,
 'kernel': 'linear'}

